In [6]:
import logging

import lightning as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger
from torch.utils.data import DataLoader, Subset

from neural_bandits.bandits.linear_ucb_bandit import LinearUCBBandit
from neural_bandits.benchmark.datasets.statlog import StatlogDataset

from neural_bandits.benchmark.environment import BanditBenchmarkEnvironment
from neural_bandits.benchmark.logger_decorator import OnlineBanditLoggerDecorator

In [ ]:
dataset = StatlogDataset()
print(dataset.context_size)
print(len(dataset))

In [ ]:
from neural_bandits.utils.selectors import EpsilonGreedySelector

train_loader = DataLoader(Subset(dataset, range(5000)), batch_size=32, shuffle=True)

accelerator = "cpu"
env = BanditBenchmarkEnvironment(train_loader, device=accelerator)

bandit_module = LinearUCBBandit(
    n_features=dataset.context_size * dataset.num_actions,
    selector=EpsilonGreedySelector(0.1),
).to(accelerator)


logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.FATAL)
logger = OnlineBanditLoggerDecorator(CSVLogger("logs", name="linear_bandit", flush_logs_every_n_steps=100))

In [ ]:
for contextualized_actions in env:
    chosen_actions, _ = bandit_module.forward(contextualized_actions)

    trainer = pl.Trainer(
        max_epochs=1,
        logger=logger,
        log_every_n_steps=1,
        enable_progress_bar=False,
        enable_model_summary=False,
        accelerator=accelerator,
    )
    feedback = env.get_feedback(chosen_actions)
    batch_regret = env.compute_regret(chosen_actions)
    logger.pre_training_log({"regret": batch_regret.sum().item()})

    batch_dataloader = DataLoader(feedback, batch_size=32)
    trainer.fit(bandit_module, batch_dataloader)
    # Because of this: https://github.com/Lightning-AI/pytorch-lightning/issues/10294,
    # we need to move the model to the desired device.
    bandit_module = bandit_module.to(accelerator)

In [ ]:
# load metrics from the logger and plot
import pandas as pd
import numpy as np

metrics_csv = logger.log_dir + "/metrics.csv"
print(metrics_csv)
metrics = pd.read_csv(metrics_csv)
cumulative_reward = np.cumsum(metrics["reward"][:5000])
cumulative_regret = np.cumsum(metrics["regret"][:5000].dropna())

In [ ]:
import matplotlib.pyplot as plt

plt.plot(cumulative_reward, label="reward")
plt.plot(cumulative_regret, label="regret")
plt.xlabel("steps")
plt.ylabel("cumulative reward/regret")
plt.legend()
plt.show()

In [ ]:
# average reward
print(sum(metrics["reward"][:100]) / 100)
print(sum(metrics["reward"][:1000]) / 1000)
print(sum(metrics["reward"][:10000]) / 10000)
print(sum(metrics["regret"][:100].dropna()) / 100)
print(sum(metrics["regret"][:1000].dropna()) / 1000)
print(sum(metrics["regret"][:10000].dropna()) / 10000)